In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
#select works to select from an item
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [5]:
#To simulate the delay
import random
import time
start_time = time.time()
normal_delay = random.normalvariate(2, 0.5)
time.sleep(normal_delay)    
# print("--- %.5f seconds ---" % (time.time() - start_time))

def delay(t):
    normal_delay = random.normalvariate(t, 0.5)
    time.sleep(normal_delay)

In [3]:
driver = webdriver.Chrome('D:\ChromeDriver\chromedriver.exe')

In [4]:
driver.get('https://www.amazon.com/RockBirds-Tactical-Flashlights-Fluorescent-Ring/dp/B00X61AJYM/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1')

In [6]:
# spot [see all customer reviews]
all_cust_reviews_div = driver.find_element_by_id('reviewSummary')
all_cust_reviews_link = driver.find_element_by_id('dp-summary-see-all-reviews')
all_cust_reviews_link.click()

In [7]:
# current page to HTML
all_cust_reviews_page = driver.page_source
all_cust_reviews_soup = BS(all_cust_reviews_page, 'html5lib')

In [8]:
#get div containing all reviews body
reviews_soup_div = all_cust_reviews_soup.find('div', attrs={'id':'cm_cr-review_list'})
reviews_soup_div_data = reviews_soup_div.find_all('div', attrs={'class': 'a-section review'})
num_of_reviews = len(reviews_soup_div_data)
# for loop through reviews_soup_div_data
score_list = []
title_list = []
review_list = []
date_list =[]
for i in range(0, num_of_reviews):
    review = reviews_soup_div_data[i].find('div', attrs={'class': 'a-section celwidget'})
    # retrieve title score and append it to score_list
    score = review.find('a', attrs={'class': 'a-link-normal'})['title']
    score_list.append(score)
    # retrieve title and append it to title_list
    title = review.find('a', attrs={'data-hook': 'review-title'}).get_text()
    title_list.append(title)
    # get review data and store in review_list
    review_body = review.find('span', attrs={'data-hook': 'review-body'}).get_text()
    review_list.append(review_body)
    # retrieve date
    date = review.find('span', attrs={'data-hook': 'review-date'}).get_text().encode("utf-8")
    date_list.append(date)

In [11]:
reviews = pd.DataFrame({'Title':title_list,'Review_Content':review_list,'Score':score_list,'Date':date_list})
reviews[:5]

,Date,Review_Content,Score,Title
0,"b'on March 4, 2017'",1:580:00This video is not intended for all aud...,5.0 out of 5 stars,Quality made and a fantastic deal! Let me sho...
1,"b'on February 10, 2018'","Given my many prior, poor experiences with sev...",4.0 out of 5 stars,"Miniature in size & price, but big on light ou..."
2,"b'on March 7, 2017'",0:110:00This video is not intended for all aud...,5.0 out of 5 stars,Heavy Duty and Bright
3,"b'on September 13, 2017'","These ended up being very handy, but I didn't ...",4.0 out of 5 stars,"Small, but mighty"
4,"b'on August 25, 2017'",Handy little flashlights and I thought the cli...,4.0 out of 5 stars,"One Bright, One Dim!"


In [33]:
#import json
#reviews.to_json=('D:/df_reviews.json')

In [10]:
#df_reviews= pd.read_json('D:/reviews.json',header=0)
df_reviews[:5]

,Title,Score,Date,Review_Content
0,It isn't very bright but it's small and will e...,5.0 out of 5 stars,"on April 17, 2018",It isn't very bright but it's small and will e...
1,"Bright, but eats batteries",5.0 out of 5 stars,"on April 16, 2018","Very bright, I work in a run down steel mill a..."
2,They work pretty good. Didn't blow me away,4.0 out of 5 stars,"on April 16, 2018","They work pretty good. Didn't blow me away, an..."
3,Four Stars,4.0 out of 5 stars,"on April 16, 2018","I will only say for now so far so good, will s..."
4,Five Stars,5.0 out of 5 stars,"on April 16, 2018","Great little flashlight, very bright, will buy..."


In [12]:
## clean the score and date data 
df_reviews['Date']=pd.to_datetime(df_reviews['Date'])
df_reviews[:5]

,Title,Score,Date,Review_Content
0,It isn't very bright but it's small and will e...,5.0 out of 5 stars,2018-04-17,It isn't very bright but it's small and will e...
1,"Bright, but eats batteries",5.0 out of 5 stars,2018-04-16,"Very bright, I work in a run down steel mill a..."
2,They work pretty good. Didn't blow me away,4.0 out of 5 stars,2018-04-16,"They work pretty good. Didn't blow me away, an..."
3,Four Stars,4.0 out of 5 stars,2018-04-16,"I will only say for now so far so good, will s..."
4,Five Stars,5.0 out of 5 stars,2018-04-16,"Great little flashlight, very bright, will buy..."


In [13]:
df_reviews['Score'].replace('5.0 out of 5 stars',5.0,inplace=True)
df_reviews['Score'].replace('4.0 out of 5 stars',4.0,inplace=True)
df_reviews['Score'].replace('3.0 out of 5 stars',3.0,inplace=True)
df_reviews['Score'].replace('2.0 out of 5 stars',2.0,inplace=True)
df_reviews['Score'].replace('1.0 out of 5 stars',1.0,inplace=True)
df_reviews[:5]

,Title,Score,Date,Review_Content
0,It isn't very bright but it's small and will e...,5.0,2018-04-17,It isn't very bright but it's small and will e...
1,"Bright, but eats batteries",5.0,2018-04-16,"Very bright, I work in a run down steel mill a..."
2,They work pretty good. Didn't blow me away,4.0,2018-04-16,"They work pretty good. Didn't blow me away, an..."
3,Four Stars,4.0,2018-04-16,"I will only say for now so far so good, will s..."
4,Five Stars,5.0,2018-04-16,"Great little flashlight, very bright, will buy..."


In [14]:
df_reviews['Score'] = df_reviews['Score'].astype('int')
df_reviews['Score'].dtype

dtype('int32')

In [15]:
df_reviews['Review_Content'].dtype

dtype('O')

In [16]:
df_reviews['Score'].value_counts()

5    549
4    118
1     81
3     53
2     52
Name: Score, dtype: int64

In [17]:
df_reviews.describe()

,Score
count,853.000000
mean,4.174678
std,1.332588
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [18]:
#add a new column called “Positivity”, where score > 3 is encoded as a 1, indicating it was positively rated.
#Otherwise, it’ll be encoded as a 0, indicating it was negatively rated.
df_reviews_1=df_reviews
df_reviews_1.dropna(inplace=True)
df_reviews_1[df_reviews_1['Score'] != 3]
df_reviews_1['Positivity'] = np.where(df_reviews_1['Score'] > 3, 1, 0)
df_reviews_1[:5]

,Title,Score,Date,Review_Content,Positivity
0,It isn't very bright but it's small and will e...,5,2018-04-17,It isn't very bright but it's small and will e...,1
1,"Bright, but eats batteries",5,2018-04-16,"Very bright, I work in a run down steel mill a...",1
2,They work pretty good. Didn't blow me away,4,2018-04-16,"They work pretty good. Didn't blow me away, an...",1
3,Four Stars,4,2018-04-16,"I will only say for now so far so good, will s...",1
4,Five Stars,5,2018-04-16,"Great little flashlight, very bright, will buy...",1


In [21]:
##let’s split our data into random training&test subsets using “Review_content” and “Positivity” columns,
#and then print out the first entry and the shape of the training set.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_reviews_1['Review_Content'], df_reviews_1['Positivity'], random_state = 0)
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry: 

 It isn't very bright but it's small and will easily fit in a pocket. If you use and loose flash lights a lot these are perfect.


X_train shape:  (639,)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vect_X_train = CountVectorizer().fit(X_train)

In [23]:
vect_X_train.get_feature_names()[::100]

['00',
 'am',
 'behind',
 'campsite',
 'completely',
 'designed',
 'enabling',
 'flashes',
 'half',
 'in',
 'layed',
 'material',
 'not',
 'patients',
 'protect',
 'replaced',
 'sheered',
 'started',
 'than',
 'types',
 'weren']

In [24]:
len(vect_X_train.get_feature_names())

2075

In [26]:
X_train_vectorized = vect_X_train.transform(X_train)
X_train_vectorized

<639x2075 sparse matrix of type '<class 'numpy.int64'>'
	with 13999 stored elements in Compressed Sparse Row format>

In [27]:
X_train_vectorized.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
##train the Logistic Regression classifier based on this feature matrix X_ train_ vectorized
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression()
model_LR.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
from sklearn.metrics import roc_auc_score
predictions_LR = model_LR.predict(vect_X_train.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions_LR))

AUC:  0.7616379310344826


#### use the coefficients for each feature (a word) to determine its weight in terms of positivity and negativity.

In [32]:
feature_names = np.array(vect_X_train.get_feature_names())
sorted_coef_index = model_LR.coef_[0].argsort()
print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs: 
['not' 'junk' 'after' 'work' 'weak' 'don' 'ok' 'thought' 'day' 'one']

Largest Coefs: 
['great' 'little' 'bright' 'battery' 'awesome' 'excellent' 'bought'
 'compact' 'price' 'big']



#### Sorting the ten smallest and ten largest coefficients, we can see the model has predicted words like “not”, “weak” and “ok” to negative reviews, and words like “great”, “bright”, and “excellent” to positive reviews. Thus, we can see that the model needs to improve due to too many non-sense words extracted for positive/negative decision. Therefore, we use Tf-idf term weighting. 

In [40]:
##Tf–idf term weighting
## self-notice! min_df can be toned for train model!!!! ==remove any words from vocabulary that appear in fewer than 3 documents.
from sklearn.feature_extraction.text import TfidfVectorizer
vect_X_train_tfidf = TfidfVectorizer(min_df = 3).fit(X_train)
len(vect_X_train_tfidf.get_feature_names())

672

In [41]:
X_train_vectorized = vect_X_train_tfidf.transform(X_train)
model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_vectorized, y_train)
predictions_tfidf = model_tfidf.predict(vect_X_train_tfidf.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions_tfidf))

AUC:  0.6413793103448276


#### The AUC score reduces in Tf-idf method by 16% with features deceasing from 2075 to 672. It is not good.

In [37]:
vect_X_train_2 = TfidfVectorizer(min_df = 10).fit(X_train)
len(vect_X_train_2.get_feature_names())

269

In [42]:
feature_names_tfidf = np.array(vect_X_train_tfidf.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
print('Smallest Tfidf: \n{}\n'.format(feature_names_tfidf[sorted_tfidf_index[:10]]))
print('Largest Tfidf: \n{}\n'.format(feature_names_tfidf[sorted_tfidf_index[:-11:-1]]))

Smallest Tfidf: 
['seemed' 'drawer' 'rockbird' 'kitchen' 'given' 'dimmer' 'wrong' 'pull'
 'myself' 'between']

Largest Tfidf: 
['powerful' 'great' 'perfect' 'awesome' 'good' 'like' 'nice' 'weak' 'junk'
 'gifts']



#### test the models by randomly selecting two reviews from data

In [45]:
print(model_tfidf.predict(vect_X_train.transform(['The product is not good, I will never buy them again','No problems at all ,strong lil lights'])))

[1 1]


In [ ]:
## Seems that the tfidf model can not correctly classified the first sense into negative group rather than positive group.
## To improve the model,we can add n-gram to the model

In [63]:
vect_X_train_ngram = CountVectorizer(min_df = 3, ngram_range = (1,5)).fit(X_train)
X_train_vectorized_ngram = vect_X_train_ngram.transform(X_train)
len(vect_X_train_ngram.get_feature_names())

1986

In [86]:
## this model's hyperparameters: min_df and ngram_range, have tested many times and the result of AUC is the highest.
## The process of hyperparameters adjusting has been omited. 
model_ngram = LogisticRegression()
model_ngram.fit(X_train_vectorized_ngram, y_train)
predictions_ngram = model_ngram.predict(vect_X_train_ngram.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions_ngram))

AUC:  0.7645114942528736


In [104]:
## adjust the hyperparameters: min_df of tfidf model
vect_X_train_2 = TfidfVectorizer(min_df = 20).fit(X_train)
len(vect_X_train_2.get_feature_names())

142

In [105]:
#### this model's hyperparameters: min_df has tested many times and the result of AUC is the highest.
## The process of hyperparameters adjusting has been omited.
X_train_vectorized_2 = vect_X_train_2.transform(X_train)
model_tfidf_2 = LogisticRegression()
model_tfidf_2.fit(X_train_vectorized_2, y_train)
predictions_tfidf_2 = model_tfidf_2.predict(vect_X_train_2.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions_tfidf_2))

AUC:  0.7010057471264368


##### Compare the AUC of different model, we can conclude that based on uncleaned data of review text, ngram model with min_df = 3, ngram_range = (1,5) get the best result： AUC=0.7645

In [ ]:
#It is obvious that the low AUC score gained from three models is largely due to uncleaned text data.
## Now we try to clean the data 

In [127]:
tfidf_opimal = TfidfVectorizer(lowercase=True, norm='l2', stop_words='english', use_idf=True)
tfidf_opimal.fit_transform(X_train).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [128]:
tfidf_opimal.get_feature_names()[::100]

['00',
 'apparently',
 'bothered',
 'checking',
 'crisp',
 'dosnt',
 'features',
 'got',
 'imagined',
 'led',
 'means',
 'operate',
 'poop',
 'ready',
 'save',
 'sos',
 'switch',
 'twisted',
 'wild']

In [130]:
vect_X_train_tfidf_optimal =TfidfVectorizer (min_df = 3).fit(X_train)
len(vect_X_train_tfidf_optimal.get_feature_names())

672

In [132]:
## this method is useless....